In [1]:
from bedrock_agent.utils import get_boto_session
from bedrock_agent.tool import tool
from bedrock_agent.types import ModelConfig
from bedrock_agent.agent import BedrockAgent
from dotenv import load_dotenv
import json

load_dotenv()

session = get_boto_session()

In [2]:
@tool
def get_current_weather(city: str) -> str:
    """
    Fetches the current weather for a given city
    
    :param city: The city for which the weather is to be fetched
    """
    import random
    rng_degree = random.randrange(0, 40)
    return f"The weather in {city} is {rng_degree} degrees celsius"

@tool
def write_poem(text: str) -> dict:
    """
    Generates a poem based on the input text
    
    :param text: The text to base the poem on
    """
    return json.dumps({"text": "Roses are red, violets are blue, I am a bot, and so are you"}, indent=4)

@tool
def write_poem_summary(text: str) -> dict:
    """
    Writes a poem summary based on the input text
    
    :param text: The text to base the poem on
    """
    return text[:50]

@tool
def handoff_weather_agent(request: str) -> BedrockAgent:
    """
    Forward the request to the weather agent to fetch the weather
    
    :param request: The request to forward to the weather agent
    """
    weather_tools = [get_current_weather]
    weather_agent = BedrockAgent(
        "WeatherAgent",
        session, 
        system_message="You are a helpful agent that specializes in fetching the latest weather.",
        tools=weather_tools,
        )
    return weather_agent

@tool
def handoff_summarize_agent(request: str) -> BedrockAgent:
    """
    Forward the request to the poem agent to summarize a poem
    
    :param request: The request to forward to the poem summarization agent
    """
    poem_tools = [write_poem_summary]
    summarize_agent = BedrockAgent(
        "SummarizeAgent",
        session, 
        system_message="You are a helpful agent that specializes in summarizing poems.",
        tools=poem_tools,
        )
    return summarize_agent

@tool
def handoff_poem_agent(request: str) -> BedrockAgent:
    """
    Forward the request to the poem agent to generate a poem
    
    :param request: The request to forward to the poem agent
    """
    poem_tools = [handoff_summarize_agent]
    poem_agent = BedrockAgent(
        "PoemAgent",
        session, 
        system_message="You are a helpful agent that specializes in writing poems.",
        tools=poem_tools,
        )
    return poem_agent

In [3]:
tools = [handoff_weather_agent, handoff_poem_agent]
model_cfg = ModelConfig(model_id="amazon.nova-pro-v1:0")
agent = BedrockAgent("ExpertAgent", session, model_cfg=model_cfg, tools=tools, verbose=False)

In [4]:
res = agent.chat("Can you write a short poem about the weather in Berlin? Please also include the current weather. Please also create a summary of the poem using the specialized agent. Please think step by step and lay out a plan before acting.")

2025-01-04 21:24:00.726 | INFO     | bedrock_agent.agent:chat:170 - Agent: WeatherAgent | End of conversation. Returning response.
2025-01-04 21:24:05.131 | INFO     | bedrock_agent.agent:chat:170 - Agent: SummarizeAgent | End of conversation. Returning response.
2025-01-04 21:24:05.975 | INFO     | bedrock_agent.agent:chat:170 - Agent: PoemAgent | End of conversation. Returning response.
2025-01-04 21:24:09.643 | INFO     | bedrock_agent.agent:chat:170 - Agent: SummarizeAgent | End of conversation. Returning response.
2025-01-04 21:24:10.339 | INFO     | bedrock_agent.agent:chat:170 - Agent: PoemAgent | End of conversation. Returning response.
2025-01-04 21:24:11.960 | INFO     | bedrock_agent.agent:chat:170 - Agent: ExpertAgent | End of conversation. Returning response.


In [5]:
res = agent.chat("Can you please add another story about and include the weather of Cologne?")

2025-01-04 21:24:16.529 | INFO     | bedrock_agent.agent:chat:170 - Agent: WeatherAgent | End of conversation. Returning response.
2025-01-04 21:24:19.833 | INFO     | bedrock_agent.agent:chat:170 - Agent: PoemAgent | End of conversation. Returning response.
2025-01-04 21:24:21.367 | INFO     | bedrock_agent.agent:chat:170 - Agent: ExpertAgent | End of conversation. Returning response.


In [6]:
agent.draw_trace()


--------------------------------------------------
Message # 1
--------------------------------------------------
ROLE: USER
TEXT: Can you write a short poem about the weather in Berlin? Please also include the current weather. Please also create a summary of the poem using the specialized agent. Please think step by step and lay out a plan before acting.

--------------------------------------------------
Message # 2
--------------------------------------------------
ROLE: ASSISTANT
TEXT: <thinking> To fulfill the user's request, I need to perform the following steps:
1. Get the current weather in Berlin using the `handoff_weather_agent`.
2. Generate a poem about the weather in Berlin using the `handoff_poem_agent`.
3. Create a summary of the poem using the `handoff_poem_agent`.

Let's start by getting the current weather in Berlin. </thinking>

TOOL USE: handoff_weather_agent
INPUT: {'request': 'current weather in Berlin'}

--------------------------------------------------
Message 

In [7]:
agent.draw_handoff_agents_traces()


Handoff Agent: WeatherAgent-a688

--------------------------------------------------
Message # 1
--------------------------------------------------
ROLE: USER
TEXT: current weather in Berlin

--------------------------------------------------
Message # 2
--------------------------------------------------
ROLE: ASSISTANT
TEXT: <thinking>The user has requested the current weather in Berlin. I can use the available tool to fetch this information.</thinking>

TOOL USE: get_current_weather
INPUT: {'city': 'Berlin'}

--------------------------------------------------
Message # 3
--------------------------------------------------
ROLE: USER
TOOL RESULT: The weather in Berlin is 18 degrees celsius

--------------------------------------------------
Message # 4
--------------------------------------------------
ROLE: ASSISTANT
TEXT: The current weather in Berlin is 18 degrees Celsius. If you need more details or information about the weather in another location, just let me know!

Handoff Agen

In [8]:
print(f"Total tokens used: {agent.total_tokens_used} | total costs: {agent.total_costs} USD")

Total tokens used: 15349 | total costs: 0.0177680000 USD


In [10]:
agent.create_agent_graph()

In [13]:
# optional: store the agent graph in a json file
agent.store_agent_graph(path="../node_graph_dash/agent_graph.json")

In [ ]:
print(json.dumps(agent.agent_graph, indent=4))